# Imports

In [1]:
import pandas as pd
import logging
logger = logging.getLogger(__name__)

In [2]:
import sys
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("auto_report.log", encoding="utf-8")
    ]
)
logger.info("Logging configured: INFO level to stdout and auto_report.log")


2025-11-03 10:35:43,876 INFO __main__: Logging configured: INFO level to stdout and auto_report.log


# Read excel

In [3]:
full_data = pd.read_excel(r"C:\Users\Massyle\Documents\auto_report\data\Enq2025_Calculs TCD Filères.xlsx")

# Prepare data

In [4]:
without_numbers = full_data.copy()
without_numbers.columns = (
    without_numbers.columns
    .str.replace(r"^\s*\d+\.?\s*", "", regex=True)
    .str.strip()
)

# Interval year
[max-2;max]

In [5]:
max_year = pd.to_numeric(without_numbers['AnneeDiplomeVerifiee'], errors='coerce').max()
logger.info(f"Max year: {max_year}")

2025-11-03 10:35:50,121 INFO __main__: Max year: 2024


In [6]:
years = pd.to_numeric(without_numbers['AnneeDiplomeVerifiee'], errors='coerce')
min_year = max_year - 2
filtered = without_numbers[(years >= min_year) & (years <= max_year)].copy()


In [7]:
total_lines = filtered['AnneeDiplomeVerifiee'].count()
logger.info(f"Total lines: {total_lines}")

2025-11-03 10:35:50,163 INFO __main__: Total lines: 1449


professionnalisation fusionne avec initial

# TCD

In [ ]:
import os
from datetime import datetime
import pandas as pd
from openpyxl.utils import get_column_letter
import pickle

# -------------------------------
# CONFIGURATION
# -------------------------------
summary_columns = [
    "Situation",
    "EmploiLieuRegionEtranger",
    "EmploiContrat",
    "EmploiFranceCadre",
    "EmploiEntrepriseTaille",
    # "Calcul_Euros_EmploiSalaireBrutAnnuelAP",
    "1erEmploiLapsPourTrouverApresDiplome",
    "EmploiCommentTrouve",
    "EmploiSecteur",
    "EmploiService"
]

GENDER_COL = "IdentiteSexeVerifie"
YEAR_COL = "AnneeDiplomeVerifiee"

assert 'filtered' in globals(), "Expected a DataFrame named 'filtered' to be defined earlier."

out_dir = r"C:\Users\Massyle\Documents\auto_report"
filename = f"Enq_TCD_{datetime.now().strftime('%Y%m%d')}.xlsx"
out_path = os.path.join(out_dir, filename)

invalid = set('[]:*?/\\')
used_names = set()

def safe_sheet_name(name: str) -> str:
    cleaned = ''.join('_' if ch in invalid else ch for ch in name)
    base = cleaned[:31] or "sheet"
    candidate = base
    i = 1
    while candidate in used_names:
        suffix = f"_{i}"
        candidate = (base[:31 - len(suffix)]) + suffix
        i += 1
    used_names.add(candidate)
    return candidate

# -------------------------------
# 1. BUILD DATAFRAMES IN MEMORY
# -------------------------------
sheets_dict = {}

for col in summary_columns:
    sheet_name = safe_sheet_name(col)

    if col not in filtered.columns:
        sheets_dict[sheet_name] = pd.DataFrame({"message": [f"Column '{col}' not found in DataFrame"]})
        continue

    # Base grouping logic
    if {GENDER_COL, YEAR_COL}.issubset(filtered.columns):
        temp = filtered[[col, GENDER_COL, YEAR_COL]].dropna(subset=[col])

        # Count by Year + Gender
        grouped = temp.groupby([YEAR_COL, GENDER_COL, col]).size().reset_index(name="Count")

        # Pivot to have one column per (Year, Gender)
        pivot = grouped.pivot_table(
            index=col,
            columns=[YEAR_COL, GENDER_COL],
            values="Count",
            fill_value=0
        )

        # Add per-year total (across genders)
        years = pivot.columns.get_level_values(0).unique()
        for y in years:
            pivot[(y, "Total")] = pivot.loc[:, y].sum(axis=1)

        # Sort columns by year then gender (H, F, Total)
        pivot = pivot.sort_index(axis=1, level=[0, 1])

        sheets_dict[sheet_name] = pivot

    else:
        # Simple fallback: just counts for that column
        counts = filtered[col].value_counts(dropna=False).rename_axis(col).reset_index(name="Count")
        sheets_dict[sheet_name] = counts

# -------------------------------
# 2. WRITE ALL SHEETS TO EXCEL
# -------------------------------
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    for sheet_name, df in sheets_dict.items():
        df.to_excel(writer, sheet_name=sheet_name)

        ws = writer.book[sheet_name]

        # Auto-fit columns
        df_reset = df.reset_index() if df.index.names != [None] else df
        for idx, column in enumerate(df_reset.columns, start=1):
            column_letter = get_column_letter(idx)
            values = df_reset[column].astype(str).tolist()
            max_len = max([len(column)] + [len(v) for v in values]) if values else len(column)
            ws.column_dimensions[column_letter].width = max_len + 2

print(f"Excel summary written: {out_path}")

# Sauvegarde du dictionnaire complet contenant tous les DataFrames
pickle_path = out_path.replace(".xlsx", ".pkl")
with open(pickle_path, "wb") as f:
    pickle.dump(sheets_dict, f)

print(f"DataFrames saved for reuse: {pickle_path}")


Excel summary written: C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103.xlsx
DataFrames saved for reuse: C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103.pkl


In [13]:
import pandas as pd
import pickle
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def aggregate_data(pickle_path):
    """
    Aggregate specific categories in the data before converting to percentages.
    
    Args:
        pickle_path (str): Path to the original pickle file
        
    Returns:
        dict: Aggregated data dictionary
    """
    
    # Load the original data
    logger.info(f"Loading data from {pickle_path}")
    with open(pickle_path, "rb") as f:
        sheets_dict = pickle.load(f)
    
    # Create a copy for aggregation
    aggregated_sheets = sheets_dict.copy()
    
    # 1. Aggregate EmploiLieuRegionEtranger
    if 'EmploiLieuRegionEtranger' in aggregated_sheets:
        logger.info("Aggregating EmploiLieuRegionEtranger...")
        df_region = aggregated_sheets['EmploiLieuRegionEtranger'].copy()
        
        # Define regions to keep as is
        regions_to_keep = ['Île-de-France', 'Étranger']
        
        # Get all other regions (to be summed into 'Province')
        other_regions = [region for region in df_region.index if region not in regions_to_keep]
        
        if other_regions:
            # Sum all other regions into 'Province'
            province_data = df_region.loc[other_regions].sum()
            
            # Create new dataframe with aggregated data
            # Keep the original regions that we want to preserve
            kept_data = df_region.loc[regions_to_keep]
            
            # Add the new 'Province' row
            province_df = pd.DataFrame([province_data], index=['Province'])
            
            # Combine kept regions and province
            aggregated_region = pd.concat([kept_data, province_df])
            
            # Sort index for better readability
            aggregated_region = aggregated_region.sort_index()
            
            aggregated_sheets['EmploiLieuRegionEtranger'] = aggregated_region
            
            logger.info(f"Regions aggregated: {len(other_regions)} regions combined into 'Province'")
            logger.info(f"Final regions: {list(aggregated_region.index)}")
    
    # 2. Aggregate EmploiEntrepriseTaille
    if 'EmploiEntrepriseTaille' in aggregated_sheets:
        logger.info("Aggregating EmploiEntrepriseTaille...")
        df_taille = aggregated_sheets['EmploiEntrepriseTaille'].copy()
        
        # Define categories to sum into 'Moins de 10'
        categories_to_sum = ['0', 'De 1 à 9']
        
        # Check if both categories exist
        existing_categories = [cat for cat in categories_to_sum if cat in df_taille.index]
        
        if len(existing_categories) > 0:
            # Sum the specified categories
            moins_de_10_data = df_taille.loc[existing_categories].sum()
            
            # Create new dataframe without the categories we're aggregating
            other_categories = [cat for cat in df_taille.index if cat not in categories_to_sum]
            kept_data = df_taille.loc[other_categories]
            
            # Add the new 'Moins de 10' row
            moins_de_10_df = pd.DataFrame([moins_de_10_data], index=['Moins de 10'])
            
            # Combine kept categories and new aggregated category
            aggregated_taille = pd.concat([kept_data, moins_de_10_df])
            
            # Sort index for better readability
            aggregated_taille = aggregated_taille.sort_index()
            
            aggregated_sheets['EmploiEntrepriseTaille'] = aggregated_taille
            
            logger.info(f"Categories aggregated: {existing_categories} combined into 'Moins de 10'")
            logger.info(f"Final categories: {list(aggregated_taille.index)}")
    
    return aggregated_sheets

def save_aggregated_data(aggregated_sheets, output_path):
    """
    Save the aggregated data to a new pickle file.
    
    Args:
        aggregated_sheets (dict): Aggregated data dictionary
        output_path (str): Path for the output pickle file
    """
    logger.info(f"Saving aggregated data to {output_path}")
    with open(output_path, "wb") as f:
        pickle.dump(aggregated_sheets, f)
    
    logger.info("Aggregated data saved successfully!")

def main():
    """Main function to run the aggregation process."""
    
    # Input and output paths
    input_pickle = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103_Ecole_Branche_abr.pkl"
    output_pickle = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103_aggregated.pkl"
    
    try:
        # Perform aggregation
        aggregated_data = aggregate_data(input_pickle)
        
        # Save aggregated data
        save_aggregated_data(aggregated_data, output_pickle)
        
        # Show EmploiLieuRegionEtranger changes
        if 'EmploiLieuRegionEtranger' in aggregated_data:
            print("\nEmploiLieuRegionEtranger:")
            print("Final regions:", list(aggregated_data['EmploiLieuRegionEtranger'].index))
        
        # Show EmploiEntrepriseTaille changes
        if 'EmploiEntrepriseTaille' in aggregated_data:
            print("\nEmploiEntrepriseTaille:")
            print("Final categories:", list(aggregated_data['EmploiEntrepriseTaille'].index))
        
        print(f"\nAggregated data saved to: {output_pickle}")
        
    except Exception as e:
        logger.error(f"Error during aggregation: {str(e)}")
        raise

if __name__ == "__main__":
    main()



2025-11-03 10:12:36,295 INFO __main__: Loading data from C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103_Ecole_Branche_abr.pkl
2025-11-03 10:12:36,311 INFO __main__: Saving aggregated data to C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103_aggregated.pkl
2025-11-03 10:12:36,317 INFO __main__: Aggregated data saved successfully!

Aggregated data saved to: C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103_aggregated.pkl


In [25]:
import pandas as pd
import pickle
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def convert_to_percentages(sheets_dict):
    """
    Convert aggregated data to percentages.
    
    Args:
        sheets_dict (dict): Dictionary containing aggregated DataFrames
        
    Returns:
        dict: Dictionary containing percentage DataFrames
    """
    percent_sheets = {}
    
    for name, df in sheets_dict.items():
        df = df.copy()
        
        # Find numeric columns (handles MultiIndex as well)
        numeric_cols = df.select_dtypes(include=["number"]).columns
        
        if len(numeric_cols) == 0:
            # No numeric columns -> copy as is
            percent_sheets[name] = df
            continue
        
        # Sum by column
        col_sums = df[numeric_cols].sum(axis=0)
        
        # Avoid division by zero: if sum == 0, set 0 for all values in that column
        # Calculate %: value / col_sum * 100
        # .div handles alignment on column index for MultiIndex as well
        percent_numeric = df[numeric_cols].div(col_sums).multiply(100)
        
        # Columns with sum 0 -> replace NaN/inf with 0
        zero_cols = col_sums[col_sums == 0].index
        if len(zero_cols) > 0:
            percent_numeric.loc[:, zero_cols] = 0.0
        
        # Re-inject non-numeric columns in the same order as original
        # Build final DataFrame respecting original column order
        result = pd.DataFrame(index=df.index)
        for col in df.columns:
            if col in numeric_cols:
                result[col] = percent_numeric[col]
            else:
                result[col] = df[col]
        
        # Keep same dtype of index / names etc.
        percent_sheets[name] = result
    
    return percent_sheets

def save_percentages_to_excel(percent_sheets, output_path):
    """
    Save percentage data to Excel file.
    
    Args:
        percent_sheets (dict): Dictionary containing percentage DataFrames
        output_path (str): Path for the output Excel file
    """
    from openpyxl.utils import get_column_letter
    
    logger.info(f"Saving percentage data to {output_path}")
    
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        for name, df in percent_sheets.items():
            # By default, keep the index (like for the original)
            df.to_excel(writer, sheet_name=name)
            
            # Auto-fit columns
            ws = writer.book[name]
            df_reset = df.reset_index() if df.index.names != [None] else df
            for idx, column in enumerate(df_reset.columns, start=1):
                column_letter = get_column_letter(idx)
                values = df_reset[column].astype(str).tolist()
                max_len = max([len(column)] + [len(v) for v in values]) if values else len(column)
                ws.column_dimensions[column_letter].width = max_len + 2
    
    logger.info("Percentage Excel file saved successfully!")

def main():
    """Main function to convert aggregated data to percentages."""
    
    # Input and output paths
    input_pickle = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated.pkl"
    output_excel = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated_percent.xlsx"
    output_pickle = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated_percent.pkl"
    
    try:
        # Load aggregated data
        logger.info(f"Loading aggregated data from {input_pickle}")
        with open(input_pickle, "rb") as f:
            aggregated_sheets = pickle.load(f)
        
        # Convert to percentages
        logger.info("Converting to percentages...")
        percent_sheets = convert_to_percentages(aggregated_sheets)
        
        # Save percentages to Excel
        save_percentages_to_excel(percent_sheets, output_excel)
        
        # Save percentages to pickle
        logger.info(f"Saving percentage data to {output_pickle}")
        with open(output_pickle, "wb") as f:
            pickle.dump(percent_sheets, f)
        
        print("\n" + "="*60)
        print("PERCENTAGE CONVERSION SUMMARY")
        print("="*60)
        print(f"Percentage Excel saved to: {output_excel}")
        print(f"Percentage pickle saved to: {output_pickle}")
        
    except Exception as e:
        logger.error(f"Error during percentage conversion: {str(e)}")
        raise

if __name__ == "__main__":
    main()


2025-10-28 07:26:11,823 INFO __main__: Loading aggregated data from C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated.pkl
2025-10-28 07:26:11,884 INFO __main__: Converting to percentages...


2025-10-28 07:26:13,270 INFO __main__: Saving percentage data to C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated_percent.xlsx
2025-10-28 07:26:15,867 INFO __main__: Percentage Excel file saved successfully!
2025-10-28 07:26:15,872 INFO __main__: Saving percentage data to C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated_percent.pkl

PERCENTAGE CONVERSION SUMMARY
Percentage Excel saved to: C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated_percent.xlsx
Percentage pickle saved to: C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_aggregated_percent.pkl


In [26]:
# paths (doit exister pickle_path défini par ton script précédent)
# ex: pickle_path = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251020_121233.pkl"
with open(pickle_path, "rb") as f:
    sheets_dict = pickle.load(f)

percent_sheets = {}

for name, df in sheets_dict.items():
    df = df.copy()

    # trouver colonnes numériques (gère aussi MultiIndex)
    numeric_cols = df.select_dtypes(include=["number"]).columns

    if len(numeric_cols) == 0:
        # pas de numériques -> on copie tel quel
        percent_sheets[name] = df
        continue

    # somme par colonne
    col_sums = df[numeric_cols].sum(axis=0)

    # éviter division par zéro : si somme == 0, on met 0 pour toutes les valeurs de la colonne
    # calc % : valeur / col_sum * 100
    # .div accepte l'alignement sur l'index des colonnes pour MultiIndex aussi
    percent_numeric = df[numeric_cols].div(col_sums).multiply(100)

    # colonnes avec somme 0 -> remplacer NaN/inf par 0
    zero_cols = col_sums[col_sums == 0].index
    if len(zero_cols) > 0:
        percent_numeric.loc[:, zero_cols] = 0.0

    # réinjecter les colonnes non numériques dans le même ordre que l'original
    # construire DataFrame final en respectant l'ordre original des colonnes
    result = pd.DataFrame(index=df.index)
    for col in df.columns:
        if col in numeric_cols:
            result[col] = percent_numeric[col]
        else:
            result[col] = df[col]

    # conserver le même dtype d'index / noms etc.
    percent_sheets[name] = result

# écrire le fichier percent
percent_out_path = pickle_path.replace(".pkl", "_percent.xlsx")
with pd.ExcelWriter(percent_out_path, engine="openpyxl") as writer:
    for name, df in percent_sheets.items():
        # par défaut on conserve l'index (comme pour l'original)
        df.to_excel(writer, sheet_name=name)

print(f"Percent Excel written: {percent_out_path}")


Percent Excel written: C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251028_percent.xlsx


In [16]:
import pickle
from difflib import get_close_matches

# Use the same pickle_path variable already defined above or set it here
# pickle_path = r"C:\Users\Massyle\Documents\auto_report\Enq_TCD_20251103_aggregated.pkl"

with open(pickle_path, "rb") as f:
    sheets_dict = pickle.load(f)

print(f"Loaded {len(sheets_dict)} sheets.")

# Show a few keys to inspect naming
all_keys = list(sheets_dict.keys())
print("First 10 keys:")
for k in all_keys[:10]:
    print(" -", k)


def find_by_index_name(d: dict, target: str):
    matches = []
    for key, df in d.items():
        idx_name = getattr(getattr(df, "index", None), "name", None)
        if idx_name == target:
            matches.append(key)
    return matches


def find_by_loose_key(d: dict, target: str):
    t = target.lower()
    return [k for k in d.keys() if t in k.lower()]


def debug_find(target: str):
    exact = find_by_index_name(sheets_dict, target)
    loose = find_by_loose_key(sheets_dict, target)
    close = get_close_matches(target, all_keys, n=5, cutoff=0.6)

    print("\n=== Search for:", target, "===")
    if exact:
        print("Index-name matches (recommended):")
        for k in exact:
            print(" *", k)
    else:
        print("No index-name matches.")

    if loose:
        print("Loose key contains:")
        for k in loose:
            print(" -", k)

    if close:
        print("Close key matches:")
        for k in close:
            print(" ~", k)

# Targets to verify
for col in ["EmploiLieuRegionEtranger", "EmploiEntrepriseTaille"]:
    debug_find(col)



Loaded 45 sheets.
First 10 keys:
 - IM_Situation
 - IM_EmploiLieuRegionEtranger
 - IM_EmploiContrat
 - IM_EmploiFranceCadre
 - IM_EmploiEntrepriseTaille
 - IM_1erEmploiLapsPourTrouverApre
 - IM_EmploiCommentTrouve
 - IM_EmploiSecteur
 - IM_EmploiService
 - GI_Situation

=== Search for: EmploiLieuRegionEtranger ===
Index-name matches (recommended):
 * IM_EmploiLieuRegionEtranger
 * GI_EmploiLieuRegionEtranger
 * GU_EmploiLieuRegionEtranger
 * GB_EmploiLieuRegionEtranger
 * GP_EmploiLieuRegionEtranger
Loose key contains:
 - IM_EmploiLieuRegionEtranger
 - GI_EmploiLieuRegionEtranger
 - GU_EmploiLieuRegionEtranger
 - GB_EmploiLieuRegionEtranger
 - GP_EmploiLieuRegionEtranger
Close key matches:
 ~ IM_EmploiLieuRegionEtranger
 ~ GU_EmploiLieuRegionEtranger
 ~ GP_EmploiLieuRegionEtranger
 ~ GI_EmploiLieuRegionEtranger
 ~ GB_EmploiLieuRegionEtranger

=== Search for: EmploiEntrepriseTaille ===
Index-name matches (recommended):
 * IM_EmploiEntrepriseTaille
 * GI_EmploiEntrepriseTaille
 * GU_Empl